In [ ]:
%load_ext autoreload
%autoreload 2

import psycopg2
import pandas as pd
from poi_classifier import OSMPOIClassifier, query_nearby_pois, query_closest_transit, query_closest_transit_parallel
from pathlib import Path

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="osm",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()

In [ ]:
cur = conn.cursor()
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'planet_osm_point' ORDER BY column_name")
print([r[0] for r in cur.fetchall()])
cur.close()

In [ ]:
df = pd.read_parquet(r"C:\Users\erike\OneDrive\Dokumente\Uni Tuebingen\ML Master\1. Semester\Data Literacy\escooter-gbfs-tracker\events_with_flags.parquet")

mask = ((df['first_seen'] == True) | (df['disappeared'] == True)) & ((df['is_maintenance'] == False) & (df['is_id_reset'] == False) & (df['is_temporary_disappearance'] == False))
events = df.loc[mask, ['id', 'lon', 'lat']]
classifier = OSMPOIClassifier(config_path=Path("osm_poi_config.yaml"))

In [ ]:
df

In [ ]:
import pandas as pd
from pathlib import Path
from poi_classifier import OSMPOIClassifier, query_nearby_pois

classifier = OSMPOIClassifier(config_path=Path("osm_poi_config.yaml"))

coords = pd.DataFrame({'lon': [9.0514], 'lat': [48.5194]})
pois = query_nearby_pois(conn, coords, radius_m=100, classifier=classifier)
print(pois[['key', 'subclass', 'class', 'name', 'distance_m']].sort_values('distance_m'))

In [ ]:
pois = query_nearby_pois(conn, events, radius_m=100, classifier=classifier, id_col='id', batch_size=20000)

In [ ]:
pois[pois["class"] == "building_other"]

In [ ]:
for col in ['key', 'subclass', 'source', 'class']:
    pois[col] = pois[col].astype('category')

In [ ]:
pois.info(memory_usage='deep')

In [ ]:
pois.to_parquet('pois.parquet', compression='gzip')

In [ ]:
pois

In [ ]:
pois = pd.read_parquet('pois.parquet')

In [ ]:
db_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "osm",
    "user": "postgres",
    "password": "postgres"
}

closest_transit = query_closest_transit_parallel(
    db_params,
    events, 
    radius_m=2000, 
    batch_size=100, 
    id_col="id"
)

In [ ]:
closest_transit

In [ ]:
closest_transit.transit_mode.value_counts()

In [ ]:
closest_transit = closest_transit.rename(columns={
    'distance_m': 'closest_public_transport_distance_m',
    'transit_mode': 'closest_public_transport_mode',
    'name': 'closest_public_transport_name'
})

In [ ]:
df = df.merge(
    closest_transit[['event_id', 'closest_public_transport_distance_m', 'closest_public_transport_mode', 'closest_public_transport_name']],
    left_on='id',
    right_on='event_id',
    how='left'
).drop(columns='event_id')

In [ ]:
df

In [ ]:
closest_transit.closest_public_transport_mode = closest_transit.closest_public_transport_mode.astype("category")

In [ ]:
closest_transit.closest_public_transport_mode.value_counts()

In [ ]:
closest_transit.to_parquet('closest_transit.parquet', compression='gzip')

In [ ]:
closest_transit.info(memory_usage='deep')

In [ ]:
df.to_parquet("vehicle_events_export.parquet")

In [ ]:
df

In [ ]:
df = pd.read_parquet(r"C:\Users\erike\OneDrive\Dokumente\Uni Tuebingen\ML Master\1. Semester\Data Literacy\escooter-gbfs-tracker\vehicle_events_export.parquet")

In [ ]:
df[df.provider == "dott_bregenz"]